# ショットガンの検討
これまでにわかったこと
- 出来高（tick数）が多いほどテクニカル分析が効きやすそう
- 1銘柄をずっと持ち続けた場合とreturnは大きく変わらない?
- 最終的には目で見ての判断がまだ必要か

これから検証すること
- パラメータのチューニング（短期・中期・長期、最大保有期間)
- 売買システムの構築

In [ ]:
from datetime import datetime
from pathlib import Path
import csv

import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

import stock

pio.renderers.default = "notebook"

In [ ]:
DAILY_DATA_DIR = Path.home() / "remote/gdrive/stock/data/daily/"
TARGET_LIST_CSV = stock.DATA_DIR / "margin_trade_target_list.csv"
daily_data_files = sorted(DAILY_DATA_DIR.glob("20220218_*.csv"))

with open(TARGET_LIST_CSV) as f:
    reader = csv.reader(f)
    next(reader)
    target_list = [row[0] for row in reader]

In [ ]:
shotgun_params = stock.autotrade.shotgun.ShotgunParams(
    short_term = 5,
    mid_term= 20,
    long_term = 60,
    max_hold_days = 9
)
shotgun = stock.autotrade.shotgun.Shotgun(shotgun_params)

In [ ]:
code = target_list[5]
df = pd.read_csv(DAILY_DATA_DIR / f"20220218_{code}.csv")
results = shotgun.run(df, with_plot=True)
profit = [r.profit for r in results]
print(f"profit: {sum(profit)}")
for r in results:
    print("buy = {}, sell = {}, profit = {:.2f},  buy = {:.1f},  sell = {:.1f}".format(
        df["day"][r.buy_index], df["day"][r.sell_index], r.profit, r.buy, r.sell
    ))

In [ ]:
## 条件を満たした場合に売り -> 翌日朝に買い直しのほうが良いのでは？